<a href="https://colab.research.google.com/github/Programmerryoki/BERT_Model_For_Fake_News_Detection/blob/main/Train_BERT_v3_multiple_inputs_not_combining_all_together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 3.4MB 7.7MB/s 
     |████████████████████████████████| 1.1MB 9.3MB/s 
     |████████████████████████████████| 358kB 27.5MB/s 
     |████████████████████████████████| 37.6MB 74kB/s 
     |████████████████████████████████| 645kB 29.7MB/s 
     |████████████████████████████████| 1.2MB 38.0MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 174kB 43.9MB/s 
     |████████████████████████████████| 706kB 42.7MB/s 


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
fake_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Fake News/Fake.csv")
fake_df["label"] = 0
print(fake_df.dtypes)
true_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Fake News/True.csv")
true_df["label"] = 1
print(true_df.dtypes)

title      object
text       object
subject    object
date       object
label       int64
dtype: object
title      object
text       object
subject    object
date       object
label       int64
dtype: object


In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
train_fake_df, test_fake_df = train_test_split(fake_df, train_size=0.7, shuffle=True, random_state=2021)
train_true_df, test_true_df = train_test_split(true_df, train_size=0.7, shuffle=True, random_state=2021)
train_df = pd.concat([train_fake_df, train_true_df]).sample(frac=1).reset_index(drop=True)
test_df = pd.concat([test_fake_df, test_true_df]).sample(frac=1)
validate_df, test_df = train_test_split(test_df, train_size=0.1, shuffle=True, random_state=2021)

train_ds = df_to_dataset(train_df, True)
test_ds = df_to_dataset(test_df, True)
val_ds = df_to_dataset(validate_df, True)

print(train_ds)
print(test_ds)
print(val_ds)

<BatchDataset shapes: ({title: (None,), text: (None,), subject: (None,), date: (None,)}, (None,)), types: ({title: tf.string, text: tf.string, subject: tf.string, date: tf.string}, tf.int64)>
<BatchDataset shapes: ({title: (None,), text: (None,), subject: (None,), date: (None,)}, (None,)), types: ({title: tf.string, text: tf.string, subject: tf.string, date: tf.string}, tf.int64)>
<BatchDataset shapes: ({title: (None,), text: (None,), subject: (None,), date: (None,)}, (None,)), types: ({title: tf.string, text: tf.string, subject: tf.string, date: tf.string}, tf.int64)>


In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
def build_classifier_model():
  inputs_list = ["title", "text", "subject", "date"]
  inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string, name=IL) for IL in inputs_list]
  preprocessing_layer_lists = [hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing') for i in range(len(inputs_list))]
  # preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  # print(preprocessing_layer)
  # encoder_inputs = preprocessing_layer(inputs[0])
  # print(preprocessing_layer)
  # encoder_inputs = encoder_inputs(inputs[1])
  encoder_inputs_list = [preprocessing_layer_lists[i](inputs[i]) for i in range(len(inputs_list))]
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  print(encoder)
  outputs = encoder(encoder_inputs_list[0])
  outputs = outputs(encoder_inputs_list[1])
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(inputs, net)

model = build_classifier_model()
model.summary()
tf.keras.utils.plot_model(model)

TypeError: ignored

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# loss = tf.keras.losses.BinaryCrossentropy()
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
print(train_df.dtypes, train_ds)
print(test_df.dtypes, test_ds)
print(validate_df.dtypes, val_ds)

title      object
text       object
subject    object
date       object
label       int64
dtype: object <BatchDataset shapes: ({title: (None,), text: (None,), subject: (None,), date: (None,)}, (None,)), types: ({title: tf.string, text: tf.string, subject: tf.string, date: tf.string}, tf.int64)>
title      object
text       object
subject    object
date       object
label       int64
dtype: object <BatchDataset shapes: ({title: (None,), text: (None,), subject: (None,), date: (None,)}, (None,)), types: ({title: tf.string, text: tf.string, subject: tf.string, date: tf.string}, tf.int64)>
title      object
text       object
subject    object
date       object
label       int64
dtype: object <BatchDataset shapes: ({title: (None,), text: (None,), subject: (None,), date: (None,)}, (None,)), types: ({title: tf.string, text: tf.string, subject: tf.string, date: tf.string}, tf.int64)>


In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
classifier_model = build_classifier_model()
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
print(classifier_model)

print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                              #  y=to_train["label"].astype(str).astype(int),
                               validation_data=val_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['title', 'subject', 'date'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


983/983 [==============================] - 421s 421ms/step - loss: 0.1956 - binary_accuracy: 0.8873 - val_loss: 0.0109 - val_binary_accuracy: 0.9985
Epoch 2/5
983/983 [==============================] - 413s 420ms/step - loss: 0.0015 - binary_accuracy: 0.9997 - val_loss: 0.0077 - val_binary_accuracy: 0.9993
Epoch 3/5
983/983 [==============================] - 413s 420ms/step - loss: 4.3174e-04 - binary_accuracy: 0.9999 - val_loss: 0.0095 - val_binary_accuracy: 0.9985
Epoch 4/5
983/983 [==============================] - 413s 420ms/step - loss: 2.9475e-04 - binary_accuracy: 1.0000 - val_loss: 0.0088 - val_binary_accuracy: 0.9993
Epoch 5/5
983/983 [==============================] - 411s 418ms/step - loss: 2.6186e-04 - binary_accuracy: 1.0000 - val_loss: 0.0090 - val_binary_accuracy: 0.9993


In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

379/379 [==============================] - 74s 195ms/step - loss: 0.0011 - binary_accuracy: 0.9998
Loss: 0.001111967721953988
Accuracy: 0.9998350143432617


In [ ]:
export_dir='/content/drive/My Drive/Colab Notebooks/Fake News/Model v3/'
tf.saved_model.save(classifier_model, export_dir=export_dir)

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Fake News/Model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Fake News/Model/assets
